In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import os

In [ ]:
!unzip Reuters\ Data.zip

Archive:  Reuters Data.zip
  inflating: training_crude_10011.txt  
   creating: __MACOSX/
  inflating: __MACOSX/._training_crude_10011.txt  
  inflating: training_crude_10078.txt  
  inflating: __MACOSX/._training_crude_10078.txt  
  inflating: training_crude_10080.txt  
  inflating: __MACOSX/._training_crude_10080.txt  
  inflating: training_crude_10106.txt  
  inflating: __MACOSX/._training_crude_10106.txt  
  inflating: training_crude_10168.txt  
  inflating: __MACOSX/._training_crude_10168.txt  
  inflating: training_crude_10190.txt  
  inflating: __MACOSX/._training_crude_10190.txt  
  inflating: training_crude_10192.txt  
  inflating: __MACOSX/._training_crude_10192.txt  
  inflating: training_crude_10200.txt  
  inflating: __MACOSX/._training_crude_10200.txt  
  inflating: training_crude_10228.txt  
  inflating: __MACOSX/._training_crude_10228.txt  
  inflating: training_crude_1026.txt  
  inflating: __MACOSX/._training_crude_1026.txt  
  inflating: training_crude_10268.txt  
  

In [ ]:
#### In the path it will return a list containing the names of the files and folders present in the path
files= os.listdir()

In [ ]:
txt_files=[]
for elem in files:
  if ('.txt' in elem):
    txt_files.append(elem)

In [ ]:
len(txt_files)

927

In [ ]:
with open('training_crude_10011.txt','r') as f:
  ls = f.read()

In [ ]:
target=[]
article_text=[]

for file in txt_files:

    #with open(file,'r') as f:
      #content = f.read()
    
    #article_text.append(content)

    f=open(file,encoding='latin-1')
    article_text.append(" ".join([line.strip() for line in
                                       f if line.strip()!=""]))
    if "crude" in file:
        target.append("crude")
    else:
        target.append("money")
    f.close()

In [ ]:
mydata=pd.DataFrame({'target':target,'article_text':article_text})

In [ ]:
mydata.to_csv('text_classification.csv',index=None)

In [ ]:
mydata.shape

(927, 2)

In [ ]:
mydata['article_text'][0]

'MOBIL PLANS TO OPEN OFFICE IN PEKING Mobil Oil Corp of U.S. Plans to open an office in Peking to develop oil exploration opportunities in China, the China Daily said. It quoted Mobil president Richard Tucker, currently in Peking, as saying he is optimistic about investment prospects in China and that Peking will continue to encourage foreign private businesses to invest here. It said Mobil bought 73 mln dlrs of crude oil and oil products from China in 1986 and sold it lubricant and fertiliser, but gave no more details.'

In [ ]:
mydata['target'].value_counts()

money    538
crude    389
Name: target, dtype: int64

In [ ]:
!pip install nltk

import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

KeyboardInterrupt: ignored

In [ ]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords #### List of commonly used english stopwords
from string import punctuation
from nltk.tokenize import word_tokenize

stemmer = PorterStemmer()

my_stopwords=set(stopwords.words('english')+list(punctuation))

In [ ]:
def split_into_lemmas(sentence):
    sentence=sentence.lower()
    words = word_tokenize(sentence)
    words_sans_stop=[]
    for word in words :
        if word in my_stopwords:
          continue
        else:
          stem_word = stemmer.stem(word)
          words_sans_stop.append(stem_word)
    
    return (words_sans_stop)

In [ ]:
article_text = mydata['article_text']
preprocessed_text = []

for text in article_text:
  stemmed_non_stopwords = split_into_lemmas(text)
  preprocessed_sent = " ".join(stemmed_non_stopwords)
  preprocessed_text.append(preprocessed_sent) 

In [ ]:
mydata['article_text'] = preprocessed_text

In [ ]:
mydata.head()

,target,article_text
0,crude,mobil plan open offic peke mobil oil corp u.s....
1,money,u.k. money market forecast revis show surplu b...
2,money,poehl warn dollar fall bundesbank presid karl ...
3,money,fed say set 1.5 billion dlr custom repurchas a...
4,money,turkish central bank set lira/dollar dm rate t...


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
article_train,article_test= train_test_split(mydata,test_size=0.2,random_state=2)

In [ ]:
article_train.reset_index(drop=True,inplace=True)

In [ ]:
y_train=(article_train['target']=='money').astype(int)
y_test=(article_test['target']=='money').astype(int)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
tf = CountVectorizer()
#tf= CountVectorizer(analyzer=split_into_lemmas,min_df=20,max_df=500,
                       #stop_words=my_stop)

In [ ]:
tf.fit(article_train['article_text'])

train_tf=tf.transform(article_train['article_text'])

In [ ]:
x_train_tf = pd.DataFrame(train_tf.toarray(), 
                   columns=tf.get_feature_names())

test_tf=tf.transform(article_test['article_text'])

x_test_tf=pd.DataFrame(test_tf.toarray(),
                       columns=tf.get_feature_names())

In [ ]:
x_train_tf

,00,000,0000,001,002,003,006916,007050,01,0100,011,015,02,020,026,03,036,04,043,046,05,050,055,058,059,06,060,0600,0615,063,0635,0650,066,07,0700,0730,075,08,080,0800,...,yasuhiro,yate,ye,year,yearend,yellow,yen,yergin,yesterday,yet,yeterray,yeutter,yield,yilin,yong,york,ypf,yu,yuan,yugoslav,yugoslavia,yugoslavian,yukon,yukuo,yumurtalik,zahnradfabrik,zaid,zaki,zambia,zambian,zealand,zero,zhejiang,zimbabw,zoet,zone,zubedei,zubeidi,zulia,zverev
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
737,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
738,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
739,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_tf

<741x649 sparse matrix of type '<class 'numpy.int64'>'
	with 35108 stored elements in Compressed Sparse Row format>

In [ ]:
x_train_tf.shape

(741, 649)

In [ ]:
x_test_tf.shape

(186, 6046)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

In [ ]:
knn=KNeighborsClassifier(n_neighbors=10)

In [ ]:
knn.fit(x_train_tf,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [ ]:
predictions=knn.predict(x_test_tf)

In [ ]:
accuracy_score(y_test,predictions)

0.9408602150537635

In [ ]:
clf_svm=SVC()

In [ ]:
clf_svm.fit(x_train_tf,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
accuracy_score(y_test,clf_svm.predict(x_test_tf))

0.967741935483871

In [ ]:
clf_nb=MultinomialNB()

In [ ]:
clf_nb.fit(x_train_tf,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
accuracy_score(y_test,clf_nb.predict(x_test_tf))

0.989247311827957

In [ ]:
# different kernel functions in svm : rbf , polynomial, sigmoid , tanh
# what are parameters specific to these kernels
# other parameters to tune for knn, svm , NB
